In [1]:
import cryo
import polars as pl
import binascii
import web3
import json
from eth_abi import decode

In [48]:
# The multical cantract address, but we also need ABI
MULTICALL3_ADDRESS = '0xcA11bde05977b3631167028862bE2a173976CA11'
MULTICALL3_ABI=json.loads('[{"inputs":[{"internalType":"bool","name":"requireSuccess","type":"bool"},{"components":[{"internalType":"address","name":"target","type":"address"},{"internalType":"bytes","name":"callData","type":"bytes"}],"internalType":"struct Multicall3.Call[]","name":"calls","type":"tuple[]"}],"name":"tryAggregate","outputs":[{"components":[{"internalType":"bool","name":"success","type":"bool"},{"internalType":"bytes","name":"returnData","type":"bytes"}],"internalType":"struct Multicall3.Result[]","name":"returnData","type":"tuple[]"}],"stateMutability":"payable","type":"function"}]')

UNIV2_ETH_USDC= '0xB4e16d0168e52d35CaCD2c6185b44281Ec28C9Dc'
UNIV3_WBTC_ETH = '0xCBCdF9626bC03E24f779434178A73a0B4bad62eD'

In [595]:
# Functions
def bytes_to_hexstr(b: any) -> str:
    if isinstance(b,list):
        return [bytes_to_hexstr(a) for a in b]
    return '0x' + b.hex()

def decode_outputdata_uniV2_price(b: bytes) -> list[float]:
    aggregated_data_uniV2 = decode(['(bool,bytes)[]'], b)[0]

    ethusdc_reserves_raw = aggregated_data_uniV2[0]
    [eth_bal, usdc_bal, _] = decode(['uint112','uint112','uint32'], ethusdc_reserves_raw[1])

    eth_usdc_price = (eth_bal / usdc_bal)*1e12
    print(f"ETH/USDC: {eth_usdc_price}")


def decode_outputdata_uniV2V3_price(b: bytes) -> list[float]:
    aggregated_data_uniV2V3 = decode(['(bool,bytes)[]'], b)[0]

    # UNI-V2
    ethusdc_reserves_raw = aggregated_data_uniV2V3[0]
    [eth_bal, usdc_bal, time ] = decode(['uint112','uint112','uint32'], ethusdc_reserves_raw[1])

    eth_usdc_price = (eth_bal / usdc_bal)*1e12
    print(f"ETH/USDC: {eth_usdc_price}")

    # UNI-V3
    # slot0():
    # sqrtPriceX96 uint160, tick int24, observationIndex uint16, observationCardinality uint16, observationCardinalityNext uint16, feeProtocol uint8, unlocked bool
    # 'uint160', 'int24', 'uint16', 'uint16', 'uint16', 'uint8', 'bool'

    wbtc_eth_slot0_raw = aggregated_data_uniV2V3[1]
    wbtc_eth_slot0_sqrt_ratioX96 = decode(['uint160', 'int24', 'uint16', 'uint16', 'uint16', 'uint8', 'bool'], wbtc_eth_slot0_raw[1])[0]
    wbtc_eth_price = wbtc_eth_slot0_sqrt_ratioX96**2 / 2**192 /1e10
    eth_wbtc_price = 1/wbtc_eth_price
    print(f"WBTC/ETH: { wbtc_eth_price}")
    print(f"ETH/WBTC: { eth_wbtc_price}")

    # Timestamp
    timestamp_raw = aggregated_data_uniV2V3[-1]
    timestamp = int(timestamp_raw[1].hex(),16)
    
    return [eth_usdc_price, eth_wbtc_price,timestamp]

In [539]:
# Function Signatures 4 bytes
getBlocknumber_4b = '42cbb15c'
getBloclTimestamp_4b= '0f28c97d'
getReserves_4b = '0902f1ac'
slot0_4b = '3850c7bd'

In [540]:
txs = cryo.collect('transactions', blocks=['-10:21881984'], rpc ='https://eth.merkle.io')

In [541]:
bytes_to_hexstr(txs['transaction_hash'].to_list())[:10]

['0x17185afe19659c8a5f93a2a547058db9c660805f625400dcc89c015f0b405088',
 '0x5fbf92a74814561ad37de87b5e2c1fd590211af35c7d87e5785790dbceab1830',
 '0x72abd91c2ac8a5cf8706d663c2629acb28db1771f8363469b516a8eb509393f8',
 '0x93c047dc4368b039206159771a5165f3223c6b85d1e951c058fa3000338098cd',
 '0x1ac8af088ae8f77d094366408676245819caf9bcbba06a490887596764b1261b',
 '0xd46fe78a7aa9113b89ca90444b32b243a71cc3dfb809afa8c7016e9904782e8b',
 '0x0fca8e8943eefbad34ffea7f28a18f27b2897ec6a75cd068fc523b68c00618bf',
 '0x74ad3f7a7fc8caff74fabc5933cbd9ecd32d97a0f039ab0c488f7990a184f294',
 '0xf82048f49848fd81d37187adb7de887f2e0cb85f3d2c04a85455c38cb98ffd90',
 '0x6050315ae98f0eb328c6fae15c2759b4335d9fbc7883e463c410d3e710eeca01']

In [542]:
# web3 instance, function from web3py
w3 = web3.Web3()
m3 = w3.eth.contract(address = MULTICALL3_ADDRESS, abi=MULTICALL3_ABI)

### Get Block Number

In [543]:
aggregate_calldata = [
    [
        MULTICALL3_ADDRESS,
        f'0x{getBlocknumber_4b}',
    ]
]
calldata = m3.encode_abi("tryAggregate", args=[True, aggregate_calldata])

In [544]:
# Args of the tryAggregate function
aggregate_calldata

[['0xcA11bde05977b3631167028862bE2a173976CA11', '0x42cbb15c']]

In [545]:
# calldata: this is the call data with info we want to get:
# 0xbce38bd7 = tryAggregate(bool,(address,bytes)[])
# aggregate_calldata = are the function arguments 
# address = Multicall_address, 
# bytes = Function Sign getblocknumber_4b = '42cbb15c'
# if you see the output of call data info all there

In [546]:
# This is the encoded data to use as calldata
# 0xbce38bd7 = tryAggregate(bool,(address,bytes)[])
# Multiclall Address + Block function signature
calldata

'0xbce38bd70000000000000000000000000000000000000000000000000000000000000001000000000000000000000000000000000000000000000000000000000000004000000000000000000000000000000000000000000000000000000000000000010000000000000000000000000000000000000000000000000000000000000020000000000000000000000000ca11bde05977b3631167028862be2a173976ca110000000000000000000000000000000000000000000000000000000000000040000000000000000000000000000000000000000000000000000000000000000442cbb15c00000000000000000000000000000000000000000000000000000000'

In [547]:
cryo_kwargs = {
    'rpc': 'https://eth.merkle.io',
    'blocks': ['latest'], 
}
            
eth_call_df = cryo.collect(
    'eth_calls',
    to_address = [MULTICALL3_ADDRESS],
    call_data=[calldata],
     output_format="polars",
    **cryo_kwargs,
)

In [548]:
eth_call_df['output_data'][0]

b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00 \x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00 \x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00@\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00 \x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01Npc'

In [549]:
outdata = eth_call_df['output_data'][0]

In [550]:
print(outdata.hex())

00000000000000000000000000000000000000000000000000000000000000200000000000000000000000000000000000000000000000000000000000000001000000000000000000000000000000000000000000000000000000000000002000000000000000000000000000000000000000000000000000000000000000010000000000000000000000000000000000000000000000000000000000000040000000000000000000000000000000000000000000000000000000000000002000000000000000000000000000000000000000000000000000000000014e7063


In [551]:
decoded_data =decode(['(bool,bytes)[]'], outdata)[0]


In [552]:
# decoded_data[0]

In [553]:

# this is boolean and list first is true (we set in arg above)
# decoded_data[0][1]

In [554]:
# 0x14df71c= 21886748
#  we call it from block function
int(decoded_data[0][1].hex(),16)

21917795

## From UNI-V2: USDC >> ETH/USDC price

In [555]:
# web3 instance, function from web3py
# w3 = web3.Web3()
# m3 = w3.eth.contract(address = MULTICALL3_ADDRESS, abi=MULTICALL3_ABI)

aggregate_calldata_uniV2 = [
    [
        UNIV2_ETH_USDC,
        f'0x{getReserves_4b}',
    ]
]
calldata_uniV2 = m3.encode_abi("tryAggregate", args=[False, aggregate_calldata_uniV2])

In [556]:

print(calldata_uniV2)

0xbce38bd70000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000004000000000000000000000000000000000000000000000000000000000000000010000000000000000000000000000000000000000000000000000000000000020000000000000000000000000b4e16d0168e52d35cacd2c6185b44281ec28c9dc000000000000000000000000000000000000000000000000000000000000004000000000000000000000000000000000000000000000000000000000000000040902f1ac00000000000000000000000000000000000000000000000000000000


In [557]:
cryo_kwargs = {
    'rpc': 'https://eth.merkle.io',
    'blocks': ['latest'], 
}
            
eth_call_uniV2_df = cryo.collect(
    'eth_calls',
    to_address = [MULTICALL3_ADDRESS],
    call_data=[calldata_uniV2],
     output_format="polars",
    **cryo_kwargs,
)

#### From UNI-V2: USDC >> Reserves

In [558]:
outdata_uniV2= eth_call_uniV2_df['output_data'][0]

In [559]:
#  0x will give in order 1-Hex USDC Reserve 2-Hex ETH Reserve 3-Timestamp
outdata_uniV2.hex()

'000000000000000000000000000000000000000000000000000000000000002000000000000000000000000000000000000000000000000000000000000000010000000000000000000000000000000000000000000000000000000000000020000000000000000000000000000000000000000000000000000000000000000100000000000000000000000000000000000000000000000000000000000000400000000000000000000000000000000000000000000000000000000000000060000000000000000000000000000000000000000000000000000013697be49e410000000000000000000000000000000000000000000001b0b8a6daf063db6c750000000000000000000000000000000000000000000000000000000067bcb15f'

In [560]:
decoded_data_uniV2 = decode(['(bool,bytes)[]'], outdata_uniV2)[0]

In [561]:
decoded_data_uniV2 

((True,
  b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x13i{\xe4\x9eA\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\xb0\xb8\xa6\xda\xf0c\xdblu\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00g\xbc\xb1_'),)

In [562]:
reserves_data_uniV2 = decode(['uint112','uint112','uint32'], decoded_data_uniV2[0][1])

In [563]:
reserves_data_uniV2

(21343771074113, 7982299002717641337973, 1740419423)

#### From UNI-V2: USDC >> ETH/USDC price Def

In [564]:
output_data_uniV2= eth_call_uniV2_df['output_data'][0]

In [565]:
decode_outputdata_uniV2_price(output_data_uniV2)

ETH/USDC: 2673.887694114983


#### From UNI-V3: WBTC >> WBTC/ETH price Def

##### In this case we add the function we want to call to the previous one, so we call:
- UNIV2_ETH_USDC
- UNIV3_WBTC_ETH

In [596]:
# web3 instance, function from web3py: Done
# Aggreagte call data for V2 and V3

aggregate_calldata_uniV2_V3 = [
    [
        UNIV2_ETH_USDC,
        f'0x{getReserves_4b}',
    ],
    [
        UNIV3_WBTC_ETH,
        f'0x{slot0_4b}',
    ],
    [
        MULTICALL3_ADDRESS,
        f'0x{getBloclTimestamp_4b}'
    ]

]
calldata_uniV2_V3 = m3.encode_abi("tryAggregate", args=[False, aggregate_calldata_uniV2_V3])

In [597]:
aggregate_calldata_uniV2_V3

[['0xB4e16d0168e52d35CaCD2c6185b44281Ec28C9Dc', '0x0902f1ac'],
 ['0xCBCdF9626bC03E24f779434178A73a0B4bad62eD', '0x3850c7bd'],
 ['0xcA11bde05977b3631167028862bE2a173976CA11', '0x0f28c97d']]

In [598]:
print(calldata_uniV2_V3)

0xbce38bd7000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000400000000000000000000000000000000000000000000000000000000000000003000000000000000000000000000000000000000000000000000000000000006000000000000000000000000000000000000000000000000000000000000000e00000000000000000000000000000000000000000000000000000000000000160000000000000000000000000b4e16d0168e52d35cacd2c6185b44281ec28c9dc000000000000000000000000000000000000000000000000000000000000004000000000000000000000000000000000000000000000000000000000000000040902f1ac00000000000000000000000000000000000000000000000000000000000000000000000000000000cbcdf9626bc03e24f779434178a73a0b4bad62ed000000000000000000000000000000000000000000000000000000000000004000000000000000000000000000000000000000000000000000000000000000043850c7bd00000000000000000000000000000000000000000000000000000000000000000000000000000000ca11bde05977b3631167028862be2a173976ca11000000000000000000000000000000

In [606]:
cryo_kwargs = {
    'rpc': 'https://eth.merkle.io',
    'blocks': ['-10:latest'], 
}

eth_call_uniV2_V3_df = cryo.collect(
    'eth_calls',
    to_address = [MULTICALL3_ADDRESS],
    call_data=[calldata_uniV2_V3],
     output_format="polars",
    **cryo_kwargs,
)

In [607]:
eth_call_uniV2_V3_df

block_number,contract_address,call_data,output_data,chain_id
u32,binary,binary,binary,u64
21917818,"b""\xca\x11\xbd\xe0Yw\xb3c\x11g\x02\x88b\xbe*\x179v\xca\x11""","b""\xbc\xe3\x8b\xd7\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00""…","b""\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x20\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00""…",1
21917819,"b""\xca\x11\xbd\xe0Yw\xb3c\x11g\x02\x88b\xbe*\x179v\xca\x11""","b""\xbc\xe3\x8b\xd7\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00""…","b""\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x20\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00""…",1
21917820,"b""\xca\x11\xbd\xe0Yw\xb3c\x11g\x02\x88b\xbe*\x179v\xca\x11""","b""\xbc\xe3\x8b\xd7\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00""…","b""\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x20\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00""…",1
21917821,"b""\xca\x11\xbd\xe0Yw\xb3c\x11g\x02\x88b\xbe*\x179v\xca\x11""","b""\xbc\xe3\x8b\xd7\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00""…","b""\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x20\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00""…",1
21917822,"b""\xca\x11\xbd\xe0Yw\xb3c\x11g\x02\x88b\xbe*\x179v\xca\x11""","b""\xbc\xe3\x8b\xd7\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00""…","b""\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x20\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00""…",1
21917823,"b""\xca\x11\xbd\xe0Yw\xb3c\x11g\x02\x88b\xbe*\x179v\xca\x11""","b""\xbc\xe3\x8b\xd7\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00""…","b""\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x20\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00""…",1
21917824,"b""\xca\x11\xbd\xe0Yw\xb3c\x11g\x02\x88b\xbe*\x179v\xca\x11""","b""\xbc\xe3\x8b\xd7\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00""…","b""\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x20\x00\x00\x00\x00\x

In [608]:
outdata_uniV2V3 = eth_call_uniV2_V3_df['output_data'][0]

In [609]:
# output_data_V2V3.hex()
outdata_uniV2V3.hex()

'0000000000000000000000000000000000000000000000000000000000000020000000000000000000000000000000000000000000000000000000000000000300000000000000000000000000000000000000000000000000000000000000600000000000000000000000000000000000000000000000000000000000000120000000000000000000000000000000000000000000000000000000000000026000000000000000000000000000000000000000000000000000000000000000010000000000000000000000000000000000000000000000000000000000000040000000000000000000000000000000000000000000000000000000000000006000000000000000000000000000000000000000000000000000001367805947e20000000000000000000000000000000000000000000001b0e50017f9ad1f40080000000000000000000000000000000000000000000000000000000067bcb2f70000000000000000000000000000000000000000000000000000000000000001000000000000000000000000000000000000000000000000000000000000004000000000000000000000000000000000000000000000000000000000000000e000000000000000000000000000000000000916eac0e2677f3e0438c6b1c25982000000000000000000000000000000000000000

In [610]:
decode_outputdata_uniV2V3_price(outdata_uniV2V3)



ETH/USDC: 2671.751321001822
WBTC/ETH: 35.48474737513748
ETH/WBTC: 0.02818112214321846


[2671.751321001822, 0.02818112214321846, 1740419843]

In [611]:
prices = [decode_outputdata_uniV2V3_price(x) for x in eth_call_uniV2_V3_df['output_data'].to_list() ]

ETH/USDC: 2671.751321001822
WBTC/ETH: 35.48474737513748
ETH/WBTC: 0.02818112214321846
ETH/USDC: 2671.4629504900818
WBTC/ETH: 35.48474737513748
ETH/WBTC: 0.02818112214321846
ETH/USDC: 2671.4629504900818
WBTC/ETH: 35.48474737513748
ETH/WBTC: 0.02818112214321846
ETH/USDC: 2671.4629504900818
WBTC/ETH: 35.48474737513748
ETH/WBTC: 0.02818112214321846
ETH/USDC: 2671.4430507641273
WBTC/ETH: 35.48474737513748
ETH/WBTC: 0.02818112214321846
ETH/USDC: 2671.4430507641273
WBTC/ETH: 35.48474737513748
ETH/WBTC: 0.02818112214321846
ETH/USDC: 2671.4430507641273
WBTC/ETH: 35.48474737513748
ETH/WBTC: 0.02818112214321846
ETH/USDC: 2671.37641584923
WBTC/ETH: 35.48474737513748
ETH/WBTC: 0.02818112214321846
ETH/USDC: 2671.37641584923
WBTC/ETH: 35.48474737513748
ETH/WBTC: 0.02818112214321846
ETH/USDC: 2670.441451981124
WBTC/ETH: 35.48474737513748
ETH/WBTC: 0.02818112214321846


In [612]:
prices

[[2671.751321001822, 0.02818112214321846, 1740419843],
 [2671.4629504900818, 0.02818112214321846, 1740419855],
 [2671.4629504900818, 0.02818112214321846, 1740419867],
 [2671.4629504900818, 0.02818112214321846, 1740419879],
 [2671.4430507641273, 0.02818112214321846, 1740419891],
 [2671.4430507641273, 0.02818112214321846, 1740419903],
 [2671.4430507641273, 0.02818112214321846, 1740419915],
 [2671.37641584923, 0.02818112214321846, 1740419927],
 [2671.37641584923, 0.02818112214321846, 1740419939],
 [2670.441451981124, 0.02818112214321846, 1740419951]]